In [2]:
%%capture
import sys
import math
import random
import pickle
import matplotlib
import pandas as pd
import numpy as np
#import datetime
import datetime as dt
import matplotlib.pyplot as plt
import statsmodels.api as sm
import seaborn as sns
import regex as re
#from numpy import *
from scipy import stats
from scipy.stats import ttest_1samp
from datetime import timedelta, datetime, time, date
from statsmodels.regression.linear_model import RegressionResults
from sklearn.metrics import confusion_matrix, classification_report
from matplotlib import rc
from tqdm.notebook import tqdm
from cycler import cycler
tqdm().pandas()

fontsize = 12
rc('font', **{'family': 'serif', 'serif': ['Computer Modern'], 'size': fontsize})
rc('text', usetex=True)

color_list = ["#348ABD","#A60628","#7A68A6","#467821","#CF4457","#188487","#E24A33"]
style = {
  "lines.linewidth": 2.0,
  "axes.edgecolor": "#bcbcbc",
  "patch.linewidth": 0.5,
  "legend.fancybox": True,
  "axes.prop_cycle": cycler('color', color_list),
  "axes.facecolor": "#ffffff",
  "axes.labelsize": "large",
  "axes.grid": True,
  "patch.edgecolor": "#eeeeee",
  "axes.titlesize": "x-large",
  "svg.fonttype": "path"}

matplotlib.rcParams.update(style)

# Modify pandas display settings
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 30)

data_path     = "C:/Users/Stefa/Documents/Uni/Projektassistenz/Paper II SHORT TERM REVERSAL/Data/"
fin_data_path = "C:/Users/Stefa/Documents/Uni/Projektassistenz/Financial Data/"    


def load_data(file):
    df = pd.read_csv(file, encoding='utf-8-sig')
    df.Date = pd.to_datetime(df.Date)
    df = df.set_index(['Ticker', 'Date'])
    df = df.rename(columns={'Prediction value': 'prediction_value'})
    return df


## Filter for topics
def filter_topic(data, topic):
    print('Topic:', topic)
    return data[data[topic] > 0] 


def t_test(sample, parent):
    mu = np.mean(parent)
    t_value = (np.mean(sample)-mu)/(np.std(sample)/np.sqrt(sample.shape[0]))
    return t_value  


def get_zscores(x, idiosync_z_scores, rw=int(255/2)):
    idiosync = '_idiosync' if idiosync_z_scores else ''
    
    r = x.rolling(window=rw, closed='left', min_periods=int(0.9*rw))
    m = r.mean()
    s = r.std()
    z = (x-m)/s
    z.index.name = 'Date'    
    return z


# Filer for Industries
#constituents = pd.read_csv(data_path+'label_counts.csv')
#sectors = constituents.ICBIN.unique()
#confidence_levels = {0.9:1.645, 0.95:1.96, 0.96:2.05, 0.98:2.33, 0.99:2.58}

In [5]:
# Load test data with predictions
test_data_pred1 = load_data(data_path+'event_data_abn_returns_mc_to_mo_17_5h_z_val_idiosync_no_dupl_2002-2021_v2.csv')
test_data_pred2 = load_data(data_path+'event_data_abn_returns_mo_to_mc_17_5h_z_val_idiosync_no_dupl_2002-2021_v2.csv')

#test_data_pred_no_dupl = load_data(data_path+'test_data_pred_no_dupl_minimal.csv')

# Load z-scores
z_values_moc = pd.read_csv(fin_data_path+'z_values_mo_to_mc_127d_rw_idiosync_2022.csv', encoding='utf-8')      
z_values_moc.Date = pd.to_datetime(z_values_moc.Date)                                   
z_values_moc = z_values_moc.set_index('Date')
z_values_moc_t_I = z_values_moc.shift(1)

In [9]:
sentiment_df = test_data_pred1.loc[:, ['Sentiment', 'return_t']]
sentiment_df.columns = ['sentiment', 'return_mc_to_mo']
sentiment_df = pd.merge(left=sentiment_df, right=test_data_pred2.return_t, left_index=True, right_index=True, how='left')
sentiment_df = sentiment_df.rename(columns={'return_t':'return_mo_to_mc'})
#sentiment_df = sentiment_df.reset_index()

In [71]:
sentiment_df

timestamp  sentiment  return_mc_to_mo  \
Ticker Date                                                              
A      2002-01-08  2002-01-08 18:04:21.330   0.011019        -0.013745   
       2002-02-19  2002-02-19 20:30:05.523   0.999024         0.132448   
       2002-03-27  2002-03-27 00:58:32.077   0.001156         0.008571   
       2002-05-14  2002-05-14 21:05:15.404   0.001467        -0.030780   
       2002-05-15  2002-05-15 01:11:35.299   0.005038        -0.030780   
...                                    ...        ...              ...   
ZTS    2019-11-07  2019-11-07 07:07:24.140  -0.977418         0.022752   
       2019-11-22  2019-11-22 08:10:10.926  -0.001873         0.000734   
       2019-12-11  2019-12-11 18:14:40.379   0.012785        -0.004085   
       2020-02-11  2020-02-11 08:49:07.274   0.986339        -0.003034   
       2020-02-13  2020-02-13 07:19:34.839  -0.989478         0.000324   

                   return_mo_to_mc  
Ticker Date                         
A      2002-01-08        -0.058508  
       2002-02-19        -0.055281  
       2002-03-27         0.000564  
       2002-05-14         0.005093  
       2002-05-15         0.005093  
...                            ...  
ZTS    2019-11-07        -0.016730  
       2019-11-22        -0.004348  
       2019-12-11        -0.004261  
       2020-02-11         0.003008  
       2020-02-13        -0.006942  

[192346 rows x 4 columns]

In [90]:
sentiment_df = test_data_pred_no_dupl.loc[:, ['prediction_value', 'prev_close_date']]

sentiment_df.columns = ['sentiment', 'prev_close_date']
#sentiment_df = sentiment_df.reset_index()
sentiment_df = sentiment_df.sort_index()

In [92]:
sentiment_df

sentiment prev_close_date
Ticker Date                                 
A      2002-01-09   0.011019      2002-01-08
       2002-02-20   0.999024      2002-02-19
       2002-03-27   0.001156      2002-03-26
       2002-05-15   0.003253      2002-05-14
       2002-08-06  -0.982975      2002-08-05
...                      ...             ...
ZTS    2019-11-22  -0.001873      2019-11-21
       2019-12-12   0.012785      2019-12-11
       2020-02-10  -0.002295      2020-02-07
       2020-02-11   0.986339      2020-02-10
       2020-02-13  -0.989478      2020-02-12

[164741 rows x 2 columns]

In [72]:
sentiment_df.to_csv(data_path+'news_events.csv', encoding='utf-8')
#sentiment_df.to_csv(data_path+'news_events_no_dupl.csv', encoding='utf-8')

(2137258, 15)


In [47]:
# Load z-scores
zvals_returns1 = pd.read_csv(extended_model_path+'z_values_127d_mo_to_mc_idiosync_abn_returns_mc_to_mo_2002-2021.csv', encoding='utf-8-sig')   # z_values_returns_z_val_neg_95_
zvals_returns1 = zvals_returns1.rename(columns = {'Unnamed: 0':'Ticker'})
zvals_returns1.Date = pd.to_datetime(zvals_returns1.Date)
zvals_returns1 = zvals_returns1.set_index(['Ticker', 'Date'])
zvals_returns1 = zvals_returns1.sort_index()
print(zvals_returns1.shape)

zvals_returns2 = pd.read_csv(extended_model_path+'z_values_127d_mo_to_mc_idiosync_abn_returns_mo_to_mc_2002-2021.csv', encoding='utf-8-sig')   # z_values_returns_z_val_neg_95_
zvals_returns2 = zvals_returns2.rename(columns = {'Unnamed: 0':'Ticker'})
zvals_returns2.Date = pd.to_datetime(zvals_returns2.Date)
zvals_returns2 = zvals_returns2.set_index(['Ticker', 'Date'])
zvals_returns2 = zvals_returns2.sort_index()
print(zvals_returns2.shape)

(2308928, 15)
(2308915, 15)


In [48]:
z_score_df = pd.concat([zvals_returns1[['z_value', 'return_t_1']], zvals_returns2['return_t_1']], axis=1)
z_score_df.columns = ['z_score', 'return_mc_to_mo', 'return_mo_to_mc']
z_score_df

z_score  return_mc_to_mo  return_mo_to_mc
Ticker Date                                                  
A      2002-01-09 -1.269700         0.008130         0.056774
       2002-01-10  1.035020         0.002108         0.001117
       2002-01-11 -0.073116        -0.012374         0.014875
       2002-01-14  0.203211         0.011650         0.090214
       2002-01-15  1.688259        -0.013944         0.008410
...                     ...              ...              ...
ZTS    2020-01-13 -2.128586         0.001661        -0.009356
       2020-01-14 -0.453477         0.007563        -0.003571
       2020-01-15 -0.122527        -0.005327         0.003124
       2020-01-16  0.282801        -0.004754         0.022713
       2020-01-17  1.431562              NaN              NaN

[2308929 rows x 3 columns]

In [49]:
z_score_df.to_csv(data_path+'z_scores.csv', encoding='utf-8')

# 
Load Files

In [104]:
z_score_df  = pd.read_csv(data_path+'z_scores.csv', encoding='utf-8')
news_events = pd.read_csv(data_path+'news_events_no_dupl_minimal.csv', encoding='utf-8')

z_score_df.Date = pd.to_datetime(z_score_df.Date)
z_score_df = z_score_df.set_index(['Ticker', 'Date'])

news_events = news_events[['Ticker', 'sentiment', 'prev_close_date']]
news_events = news_events.rename(columns={'prev_close_date':'Date'})
news_events.Date = pd.to_datetime(news_events.Date)
news_events = news_events.set_index(['Ticker', 'Date'])
news_events = news_events.sort_index()

In [105]:
z_score_df = z_score_df.sort_index()
z_score_df['z_score_tp1'] = z_score_df.groupby(level=[0])['z_score'].shift(-1)

In [106]:
z_score_df#.loc[((slice(None), 'MSFT')), 'z_score']

z_score  return_mc_to_mo  return_mo_to_mc  z_score_tp1
Ticker Date                                                               
A      2002-01-09 -1.269700         0.008130         0.056774     1.035020
       2002-01-10  1.035020         0.002108         0.001117    -0.073116
       2002-01-11 -0.073116        -0.012374         0.014875     0.203211
       2002-01-14  0.203211         0.011650         0.090214     1.688259
       2002-01-15  1.688259        -0.013944         0.008410     0.049628
...                     ...              ...              ...          ...
ZTS    2020-01-13 -2.128586         0.001661        -0.009356    -0.453477
       2020-01-14 -0.453477         0.007563        -0.003571    -0.122527
       2020-01-15 -0.122527        -0.005327         0.003124     0.282801
       2020-01-16  0.282801        -0.004754         0.022713     1.431562
       2020-01-17  1.431562              NaN              NaN          NaN

[2308929 rows x 4 columns]

In [107]:
z_score_df.z_score_tp1.isna().sum()

119471

In [108]:
z_score_df.z_score.isna().sum()

118717

In [109]:
z_score_df.loc[z_score_df.z_score.isna()]#.sum()

z_score  return_mc_to_mo  return_mo_to_mc  z_score_tp1
Ticker Date                                                              
A      2002-01-21      NaN        -0.004086        -0.020280    -0.488986
       2002-02-18      NaN        -0.038386        -0.000522    -0.109069
       2002-03-29      NaN        -0.010978         0.020937     0.515444
       2002-04-19      NaN        -0.012449         0.025631     0.592614
       2002-05-27      NaN         0.005728         0.006684     0.182122
...                    ...              ...              ...          ...
ZTS    2019-07-04      NaN         0.010249        -0.013708    -0.843088
       2019-09-02      NaN         0.009341         0.006122     0.270176
       2019-11-28      NaN         0.002421        -0.000274     0.085196
       2019-12-25      NaN        -0.001522         0.001549     0.192400
       2020-01-01      NaN        -0.009208         0.024474     1.550449

[118717 rows x 4 columns]

In [110]:
news_events

sentiment
Ticker Date                 
A      2002-01-08   0.011019
       2002-02-19   0.999024
       2002-03-26   0.001156
       2002-05-14   0.003253
       2002-08-05  -0.982975
...                      ...
ZTS    2019-11-21  -0.001873
       2019-12-11   0.012785
       2020-02-07  -0.002295
       2020-02-10   0.986339
       2020-02-12  -0.989478

[164741 rows x 1 columns]

In [111]:
z_scores2 = pd.merge(left=z_score_df, right=news_events, left_index=True, right_index=True, how='left')

In [112]:
z_scores2#.loc[('AAPL', slice(None)),:].head(29)

z_score  return_mc_to_mo  return_mo_to_mc  z_score_tp1  \
Ticker Date                                                                  
A      2002-01-09 -1.269700         0.008130         0.056774     1.035020   
       2002-01-10  1.035020         0.002108         0.001117    -0.073116   
       2002-01-11 -0.073116        -0.012374         0.014875     0.203211   
       2002-01-14  0.203211         0.011650         0.090214     1.688259   
       2002-01-15  1.688259        -0.013944         0.008410     0.049628   
...                     ...              ...              ...          ...   
ZTS    2020-01-13 -2.128586         0.001661        -0.009356    -0.453477   
       2020-01-14 -0.453477         0.007563        -0.003571    -0.122527   
       2020-01-15 -0.122527        -0.005327         0.003124     0.282801   
       2020-01-16  0.282801        -0.004754         0.022713     1.431562   
       2020-01-17  1.431562              NaN              NaN          NaN   

                   sentiment  
Ticker Date                   
A      2002-01-09        NaN  
       2002-01-10        NaN  
       2002-01-11        NaN  
       2002-01-14        NaN  
       2002-01-15        NaN  
...                      ...  
ZTS    2020-01-13        NaN  
       2020-01-14        NaN  
       2020-01-15        NaN  
       2020-01-16        NaN  
       2020-01-17        NaN  

[2308929 rows x 5 columns]

In [113]:
z_scores2.to_csv(data_path+'z_scores_merged.csv', encoding='utf-8')

In [120]:
z_scores2.z_score.isna().sum()

118717

In [115]:
z_scores2.sentiment.notna().sum()

162170

In [118]:
z_scores2.loc[((z_scores2.z_score_tp1.notna()) & (z_scores2.sentiment.notna()))].shape

(155976, 5)

In [119]:
z_scores2.loc[((z_scores2.z_score_tp1.isna()) & (z_scores2.sentiment.notna()))].shape

(6194, 5)